In [ ]:
!pip install git+https://github.com/openai/whisper.git

import whisper
import os
import requests
import json
from google.colab import files

# Obtain API key and summarize transcription
# 到這裡取得 Gemini API  https://aistudio.google.com/app/u/8/apikey 
api_key = "Gemini API key"  # Replace with your Gemini API key
summary = summarize_with_gemini(transcribed_text, api_key)


# Load Whisper model
model = whisper.load_model("medium") # Available models: tiny, base, small, medium, large, large-v2, large-v3

# Set audio file path
audio_path = "/content/water.m4a" # Replace with your audio file name

# Transcribe audio file with Whisper (set to Traditional Chinese if needed)
result = model.transcribe(audio_path, prompt="請轉錄以下繁體中文的內容：")
transcribed_text = result["text"]

# Print transcription result
print("Transcription:")
print(transcribed_text)

# Function to summarize transcription using Gemini API
def summarize_with_gemini(transcribed_text, api_key):
    url = "https://generativelanguage.googleapis.com/v1/models/gemini-pro:generateContent"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    # Crafting prompt for summarization
    prompt = f"Summarize the following meeting transcript into concise bullet points:\n\n{transcribed_text}"
    payload = {
        "prompt": {"text": prompt}
    }
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    if response.status_code == 200:
        result = response.json()
        summary = result['contents'][0]['parts'][0]['text']
        return summary
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None



# Print and save the summary if available
if summary:
    print("Meeting Summary:")
    print(summary)

    # Save transcription and summary as text files
    file_name = os.path.splitext(os.path.basename(audio_path))[0]
    with open(f"{file_name}.txt", "w") as file:
        file.write(transcribed_text)
        files.download(f"{file_name}.txt")

    with open(f"{file_name}_summary.txt", "w") as file:
        file.write(summary)
        files.download(f"{file_name}_summary.txt")